from os.path import dirname, join as pjoin
import scipy.io as sio
from scipy.linalg import solve
from scipy.spatial import Delaunay
import plotly.figure_factory as ff
import numpy as np
import numpy.matlib
import matplotlib.pyplot as plt

# Translated libs
from meshlib import *
from rwglib import *

In [1]:
from libs_import import *

# Creating mesh

## Create 1st zone 

In [2]:
hull_bound_points = np.array([[1,-40],[-1,-40],[1,40],[-1,40]])
mesh_resolution = 2
tri = creatHull(hull_bound_points,mesh_resolution)
plt.triplot(tri.points[:,0], tri.points[:,1], tri.simplices)
plt.show()

NameError: name 'Delaunay_temp' is not defined

Using function
$$to3D(tri2D,scaleX,scaleY,rotX,rotY,rotZ,dX,dY,dZ)$$

In [ ]:
tri_final = to3D(tri,1/100,1/40,0,0,0,0,0,0)

## Save to "p" and "t"

In [ ]:
p = tri_final.points.T
t = np.ones((4,tri_final.simplices.shape[0]))
t[0:3,:] = tri_final.simplices.T + 1
t = t.astype(int)

# RWG1 Geometry calculations - all Chapters
Uses the structure mesh file, e.g. platefine.mat, as an input.

Creates the RWG edge element for every inner edge of the structure. The total number of elements is EdgesTotal.

Outputs the following arrays:

Edge first node number          

    Edge_(1,1:EdgesTotal)
   
Edge second node number  

    Edge_(2,1:EdgesTotal)
    
Plus triangle number  

    TrianglePlus(1:EdgesTotal)
   
Minus triangle number           

    TriangleMinus(1:EdgesTotal)
   
Edge length                     

    EdgeLength(1:EdgesTotal)
   
Edge element indicator          
    
    EdgeIndicator(1:EdgesTotal)

Also outputs areas and midpoints of separate triangles:

Triangle area                   

    Area(1:TrianglesTotal)
   
Triangle center                 

    Center(1:TrianglesTotal)      
   
This script may handle surfaces with T-junctions including monopoles over various metal surfaces and certain metal meshes

Copyright 2002 AEMM. Revision 2002/03/09 Chapter 2


Plot the mesh

In [ ]:
x_,y_,z_ = p
fig_scale = max(max(x_)-min(x_),max(y_)-min(y_),max(z_)-min(z_))
x_scale,y_scale,z_scale=(max(x_)-min(x_))/fig_scale,(max(y_)-min(y_))/fig_scale,(max(z_)-min(z_))/fig_scale
fig1 = ff.create_trisurf(x=x_, y=y_, z=z_,
                         simplices=(t[0:3,:].T-1),
                         color_func=[i for i in range(len(t.T))],
                         show_colorbar=False,
                         title="Antenna mesh", aspectratio=dict(x=x_scale, y=y_scale, z=0.3))
fig1.show()

In [ ]:
ANT_mesh1 = RWGmesh(p,t)

This block is only meaningful for T junctions. It leaves only two edge elements at a junction

# RWG2: Geometry calculations - all Chapters
Uses the mesh file from RWG1, mesh1.mat, as an input.

Creates the following parameters of the RWG edge elements: 

Position vector rho_c_plus from the free vertex of the "plus" triangle to its center

                                   RHO_Plus(1:3,1:EdgesTotal)
                                   
Position vector rho_c_minus from the center of the "minus" triangle to its free vertex 

                                   RHO_Minus(1:3,1:EdgesTotal)

In addition to these parameters creates the following arrays for nine subtriangles (barycentric subdivision):

Midpoints of nine subtriangles

                                   Center_(1:3,1:9,1:TrianglesTotal)  
                                   
Position vectors rho_c_plus from the free vertex of the "plus" triangle to nine subtriangle midpoints

                                   RHO__Plus(1:3,1:9,1:EdgesTotal)
                                   
Position vectors rho_c_minus from nine subtriangle midpoints to the free vertex of the "minus" triangle

                                   RHO__Minus(1:3,1:9,1:EdgesTotal)

See Rao, Wilton, Glisson, IEEE Trans. Antennas and Propagation, vol. AP-30, No 3, pp. 409-418, 1982.

Copyright 2002 AEMM. Revision 2002/03/05 
Chapter 2

In [ ]:
ANT_mesh2 = RWGmesh2(ANT_mesh1,t,p)

# RWG3 Calculates the impedance matrix using function IMPMET
   Uses the mesh file from RWG2, mesh2.mat, as an input.

The following parameters need to be specified prior to calculations:

    Frequency (Hz)                  f

    Dielectric constant (SI)        epsilon_

    Magnetic permeability (SI)      mu_

Copyright 2002 AEMM. Revision 2002/03/11 
Chapter 2

## Fixed parameters

Related to antenna feeding position

In [ ]:
FeedPoint=[[0],[0],[0]]

Speed of light & Free-space impedance

In [ ]:
frequency          = [i for i in range(10000000,500100000,1000000)]
epsilon_    = 8.854e-012
mu_         = 1.257e-006
c_=1/np.sqrt(epsilon_*mu_)
eta_=np.sqrt(mu_/epsilon_)

### No lumped component

In [ ]:
RL = np.zeros(len(frequency))
Impedance = np.zeros(len(frequency),dtype = 'complex_')
for i in range(len(frequency)):
    Z, I, Impedance[i], FeedPower = calculateImpedance(frequency[i],c_,mu_,epsilon_,p,ANT_mesh1,ANT_mesh2,FeedPoint)
    RL[i] = 10*np.log(np.abs((Impedance[i]-50)/(Impedance[i]+50)))
    #print('Return loss is ', Impedance[i], ' @',frequency[i]/1e6,'MHz')
    
plt.plot(np.array(frequency)/1e6, np.real(Impedance), '--', color='black', label='real')
plt.plot(np.array(frequency)/1e6, np.imag(Impedance), color='black', label='imaginary')
plt.grid()
plt.xlabel("Freq [MHz]") 
plt.ylabel("S11") 
plt.legend()
plt.title('Reflection coefficient')
plt.show()

## Calculate impedance with lumped components

In [ ]:
LNumber=2
LoadPoint = np.array([[0,0.50,0],[0,-0.50,0]]).T
LoadValue = np.array([[0,1e+64,100],[0,1e+64,100]]).T   #((L1,C1,R1),(L2,C2,R2))
LoadDir=np.array([[0,1,0],[0,1,0]]).T

RL = np.zeros(len(frequency))
Impedance = np.zeros(len(frequency),dtype = 'complex_')
for i in range(len(frequency)):
    Z, I, Impedance[i], FeedPower = calculateImpedance_withLumped(frequency[i],c_,mu_,epsilon_,
                                                           p,ANT_mesh1,ANT_mesh2,FeedPoint,
                                                           LNumber,LoadPoint,LoadValue,LoadDir)
    RL[i] = 10*np.log(np.abs((Impedance[i]-50)/(Impedance[i]+50)))
    #print('Return loss is ', Impedance[i], ' @',frequency[i]/1e6,'MHz')
    
plt.plot(np.array(frequency)/1e6, np.real(Impedance), '--', color='black', label='real')
plt.plot(np.array(frequency)/1e6, np.imag(Impedance), color='black', label='imaginary')
plt.grid()
plt.xlabel("Freq [MHz]") 
plt.ylabel("S11") 
plt.legend()
plt.title('Reflection coefficient loading 2x100 Ohm')
plt.show()

In [ ]:
LNumber=2
LoadPoint = np.array([[0,0.50,0],[0,-0.50,0]]).T
LoadValue = np.array([[0,1e+64,200],[0,1e+64,200]]).T   #((L1,C1,R1),(L2,C2,R2))
LoadDir=np.array([[0,1,0],[0,1,0]]).T

RL = np.zeros(len(frequency))
Impedance = np.zeros(len(frequency),dtype = 'complex_')
for i in range(len(frequency)):
    Z, I, Impedance[i], FeedPower = calculateImpedance_withLumped(frequency[i],c_,mu_,epsilon_,
                                                           p,ANT_mesh1,ANT_mesh2,FeedPoint,
                                                           LNumber,LoadPoint,LoadValue,LoadDir)
    RL[i] = 10*np.log(np.abs((Impedance[i]-50)/(Impedance[i]+50)))
    #print('Return loss is ', Impedance[i], ' @',frequency[i]/1e6,'MHz')
    
plt.plot(np.array(frequency)/1e6, np.real(Impedance), '--', color='black', label='real')
plt.plot(np.array(frequency)/1e6, np.imag(Impedance), color='black', label='imaginary')
plt.grid()
plt.xlabel("Freq [MHz]") 
plt.ylabel("S11") 
plt.legend()
plt.title('Reflection coefficient loading 2x200 Ohm')
plt.show()

In [ ]:
LNumber=2
LoadPoint = np.array([[0,0.50,0],[0,-0.50,0]]).T
LoadValue = np.array([[0,10e-12,0],[0,10e-12,0]]).T   #((L1,C1,R1),(L2,C2,R2))
LoadDir=np.array([[0,1,0],[0,1,0]]).T

RL = np.zeros(len(frequency))
Impedance = np.zeros(len(frequency),dtype = 'complex_')
for i in range(len(frequency)):
    Z, I, Impedance[i], FeedPower = calculateImpedance_withLumped(frequency[i],c_,mu_,epsilon_,
                                                           p,ANT_mesh1,ANT_mesh2,FeedPoint,
                                                           LNumber,LoadPoint,LoadValue,LoadDir)
    RL[i] = 10*np.log(np.abs((Impedance[i]-50)/(Impedance[i]+50)))
    #print('Return loss is ', Impedance[i], ' @',frequency[i]/1e6,'MHz')
    
plt.plot(np.array(frequency)/1e6, np.real(Impedance), '--', color='black', label='real')
plt.plot(np.array(frequency)/1e6, np.imag(Impedance), color='black', label='imaginary')
plt.grid()
plt.xlabel("Freq [MHz]") 
plt.ylabel("S11") 
plt.legend()
plt.title('Reflection coefficient loading 2x10 pF')
plt.show()

In [ ]:
LNumber=2
LoadPoint = np.array([[0,0.50,0],[0,-0.50,0]]).T
LoadValue = np.array([[0,1e-12,0],[0,1e-12,0]]).T   #((L1,C1,R1),(L2,C2,R2))
LoadDir=np.array([[0,1,0],[0,1,0]]).T

RL = np.zeros(len(frequency))
Impedance = np.zeros(len(frequency),dtype = 'complex_')
for i in range(len(frequency)):
    Z, I, Impedance[i], FeedPower = calculateImpedance_withLumped(frequency[i],c_,mu_,epsilon_,
                                                           p,ANT_mesh1,ANT_mesh2,FeedPoint,
                                                           LNumber,LoadPoint,LoadValue,LoadDir)
    RL[i] = 10*np.log(np.abs((Impedance[i]-50)/(Impedance[i]+50)))
    #print('Return loss is ', Impedance[i], ' @',frequency[i]/1e6,'MHz')
    
plt.plot(np.array(frequency)/1e6, np.real(Impedance), '--', color='black', label='real')
plt.plot(np.array(frequency)/1e6, np.imag(Impedance), color='black', label='imaginary')
plt.grid()
plt.xlabel("Freq [MHz]") 
plt.ylabel("S11") 
plt.legend()
plt.title('Reflection coefficient loading 2x1 pF')
plt.show()

# Calculate surface currents

In [ ]:
f = 75e6

In [ ]:
Z, I, Impedance, FeedPower = calculateImpedance(f,c_,mu_,epsilon_,p,ANT_mesh1,ANT_mesh2,FeedPoint)

In [ ]:
SCurrent = currentDistribution(t,ANT_mesh1,ANT_mesh2,I)

In [ ]:
Jmax = max(SCurrent)
CurrentNorm = SCurrent/max(SCurrent)

In [ ]:
fig2 = ff.create_trisurf(x=x_, y=y_, z=z_,
                         simplices=(t[0:3,:].T-1),
                         color_func=list(CurrentNorm),
                         show_colorbar=True,
                         title="Antenna mesh - Normalized current distribution", aspectratio=dict(x=x_scale, y=y_scale, z=0.3))
fig2.show()

# EFIELD2 Radiated/scattered field over a large sphere

Uses the mesh file from RWG2, mesh2.mat, and the file containing surface current coefficients, current.mat, from RWG4 as inputs.

Uses the structure sphere.mat/sphere1.mat to display radiation intensity distribution over the sphere surface. 

The sphere doesn't intersect the radiating object.

The following parameters need to be specified:
        
        Sphere radius (m)

Copyright 2002 AEMM. Revision 2002/03/11 
Chapter 3

In [ ]:
mat_fname_sphere = pjoin('mesh', 'sphere.mat')
print(mat_fname_sphere)
mat_contents_sphere = sio.loadmat(mat_fname_sphere)
p_sphere = mat_contents_sphere['p']
t_sphere = mat_contents_sphere['t']
p_sphere=100*p_sphere
K=1j*(2*np.pi*f/c_)

In [ ]:
ANT_moment = RWGmoment(ANT_mesh1,I)

In [ ]:
U, TotalPower = radiating3DPower(t_sphere,p_sphere,ANT_moment,K,eta_)

In [ ]:
GainLogarithmic = 10*np.log10(4*np.pi*max(U)/TotalPower)

In [ ]:
U_norm = 10*np.log10(U/np.linalg.norm(U))
x_sphere,y_sphere,z_sphere = p_sphere
fig_scale_sphere = max(max(x_sphere)-min(x_sphere),max(y_sphere)-min(y_sphere),max(z_sphere)-min(z_sphere))
x_scale_sphere,y_scale_sphere,z_scale_sphere=(max(x_sphere)-min(x_sphere))/fig_scale_sphere,(max(y_sphere)-min(y_sphere))/fig_scale_sphere,(max(z_sphere)-min(z_sphere))/fig_scale_sphere
fig3 = ff.create_trisurf(x=x_sphere, y=y_sphere, z=z_sphere,
                         simplices=(t_sphere[0:3,:].T-1),
                         color_func=U_norm,
                         show_colorbar=True,
                         title="Gain", aspectratio=dict(x=x_scale_sphere, y=y_scale_sphere, z=z_scale_sphere))
fig3.show()

# EFIELD3 2D Radiation patterns

Uses the mesh file from RWG2, mesh2.mat, and the file containing surface current coefficients, current.mat, from RWG4 as inputs.

Additionally uses the value of TotalPower saved  in file gainpower.mat (script efield2.m)

The following parameters need to be specified:

    Radius of the circle (m)            R
    Plane of the circle:                [x y 0] or 
                                        [x 0 z] or 
                                        [0 y z] 
Number of discretization points per pattern                             

                                        NumPoints

Copyright 2002 AEMM. Revision 2002/03/11 

Chapter 3

In [ ]:
NumPoints = 100
R = 1000 # meter - distance to the antenna

Fix Phi and scan Theta to have 2D pattern

In [ ]:
theta = np.array([2*np.pi*(ii)/(NumPoints -1) for ii in range(NumPoints)])
phi = 0*np.pi
ObservationPointList_phi0 = [np.array([R*np.sin(theta_)*np.cos(phi), R*np.sin(theta_)*np.sin(phi), R*np.cos(theta_)]) for theta_ in theta]
phi = 0.5*np.pi
ObservationPointList_phi90 = [np.array([R*np.sin(theta_)*np.cos(phi), R*np.sin(theta_)*np.sin(phi), R*np.cos(theta_)]) for theta_ in theta]

Calculate the E and H fields in the prepared cuts

In [ ]:
U_efield3_phi0, W_efield3_phi0 = radiating2DFields(ObservationPointList_phi0,ANT_moment,K,eta_)
U_efield3_phi90, W_efield3_phi90 = radiating2DFields(ObservationPointList_phi90,ANT_moment,K,eta_)

In [ ]:
Polar_0 = 10*np.log10(4*np.pi*U_efield3_phi0/TotalPower)
Polar_90 = 10*np.log10(4*np.pi*U_efield3_phi90/TotalPower)
OFFSET = -20
fig4, ax = plt.subplots(subplot_kw={'projection': 'polar'})
ax.plot(theta, np.where(Polar_0 < OFFSET, OFFSET, Polar_0), color='red', label='Phi = 0°')
ax.plot(theta, np.where(Polar_90 < OFFSET, OFFSET, Polar_90), color='blue', label='Phi = 90°')
ax.legend()
ax.set_rmin(OFFSET)
ax.set_rticks([-15, -10, -5, 0])  # Less radial ticks
#ax.set_rlabel_position(-22.5)  # Move radial labels away from plotted line
ax.grid(True)

ax.set_title("E-field pattern in Phi = 0° and 90° plane", va='bottom')
plt.show()

In [ ]:
print("Max directivity of the antenna is",max(Polar_90),"dBi")

In [ ]:
theta_ = (np.arange(NumPoints)/(NumPoints-1))*np.pi
phi_ = (-1/2 + np.arange(NumPoints)/(NumPoints-1))*np.pi*2
Theta, Phi = np.meshgrid(theta_,phi_)
#Field calculation 
FF_3D = np.zeros(Theta.shape)
for i in range(NumPoints):
    ObservationPointList = [np.array([R*np.sin(theta__)*np.cos(phi_[i]), R*np.sin(theta__)*np.sin(phi_[i]), R*np.cos(theta__)]) for theta__ in theta_]
    FF_3D[i,:], W_efield3 = radiating2DFields(ObservationPointList,ANT_moment,K,eta_)
        
Pattern3D(Theta, Phi, FF_3D)